In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import optim
from s4hci.utils.data_handling import load_adi_data
from applefy.utils.file_handling import open_fits, save_as_fits
from applefy.utils.fake_planets import add_fake_planets

In [2]:
from s4hci.models.psf_subtraction import S4

In [3]:
torch.set_float32_matmul_precision("high")

# Load the data and insert the fake planet

In [4]:
# 2.) Load the dataset
science_data, raw_angles, raw_psf_template_data = \
    load_adi_data(
        hdf5_dataset="/fast/mbonse/s4/30_data/HD22049_303_199_C-0065_C_.hdf5",
        data_tag="object",
        psf_template_tag="psf_template",
        para_tag="header_object/PARANG")

science_data = science_data[:, 35:-35, 35:-35]
raw_angles = raw_angles[:]

# Background subtraction of the PSF template
psf_template_data = np.median(raw_psf_template_data, axis=0)
psf_template_data = psf_template_data - np.min(psf_template_data)

In [5]:
np.max(psf_template_data)

10764.111528773727

# Fit the FWHM

In [6]:
from astropy.modeling.functional_models import Moffat2D
from astropy.modeling import fitting

In [10]:
def get_fwhm(psf_template_data):
    # Fit the data using astropy.modeling
    p_init = Moffat2D(
        amplitude=np.max(psf_template),
        x_0=psf_template_data.shape[0]/2, 
        y_0=psf_template_data.shape[1]/2)
    fit_p = fitting.LevMarLSQFitter()

    y, x = np.mgrid[:psf_template.shape[0],
                    :psf_template.shape[1]]
    p = fit_p(p_init, x, y, psf_template)
    fwhm = np.round(p.fwhm, 1)
    
    return fwhm

In [11]:
get_fwhm(psf_template_data)

3.6